In [1]:
import os
from graphviz import Source
from sklearn.datasets import load_iris
from sklearn.tree import DecisionTreeClassifier, export_graphviz
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt

#To make the notebook's output stabel across runs
np.random.seed(42)

#Uses Jupyter's own backend to plot
%matplotlib inline

#To make pretty figures
mpl.rc("axes", labelsize=14)
mpl.rc("xtick", labelsize=12)
mpl.rc("ytick", labelsize=12)

#Path to saving images
IMAGE_PATH = os.path.join("images")
os.makedirs(IMAGE_PATH, exist_ok=True)

def save_fig(fig_id, tight_layout=True, fig_extension="png", resolution=300):
    path = os.path.join(IMAGE_PATH, fig_id + "." + fig_extension)
    print("Saving figure", fig_id)
    if tight_layout:
        plt.tight_layout()
    plt.savefig(path, format=fig_extension ,dpi=resolution)

ModuleNotFoundError: No module named 'graphviz'

Decision Trees can perform both classification, regression tasks and even multioutput tasks. Furthermore, they're capable of fitting complex datasets. <br>
We'll be starting with how to train -> visualize -> predictions with Decision Trees. Then go through the CART training algorithm used by Scikit Learn -> regularize trees -> use trees for regression tasks. <br>
Finally, we'll have a look at the limitation of Decision Trees

# Section: Training And Visualizing A Decision Tree

The following trains a DecisionTreeClassifier on the iris dataset

In [ ]:
iris = load_iris()
X = iris.data[:, :2]
y = iris.target

tree_clf = DecisionTreeClassifier(max_depth=2, random_state=42)
tree_clf.fit(X, y)

Let's visualize the trained Decision Tree by using the export_graphviz() method. It outputs a filled called <em>iris_tree.dot</em>

In [ ]:
export_graphviz(
    tree_clf,
    out_file = os.path.join(IMAGE_PATH, "iris_tree.dot"),
    feature_names = iris.feature_names[2:],
    class_names = iris.target_names,
    rounded = True,
    filled = True
)

Source.from_file(os.path.join(IMAGE_PATH, "iris_tree.dot"))